# Continuation basics

Let's start by giving the `Functor`, `Applicative` and `Monad` instances for the continuation type constructor. Using an infix operator instead of the standard `runCont` helps reduce boilerplate.

In [31]:
{-# LANGUAGE InstanceSigs #-}

newtype Cont r a = Cont { (>>-) :: (a -> r) -> r }

instance Functor (Cont r) where
    fmap :: (a -> b) -> Cont r a -> Cont r b
    fmap f m = Cont $ \k -> m >>- (k . f)
    
instance Applicative (Cont r) where
    pure :: a -> Cont r a
    pure a = Cont $ \k -> k a
    (<*>) :: Cont r (a -> b) -> Cont r a -> Cont r b
    m <*> n = Cont $ \k -> m >>- \f -> n >>- \x -> k $ f x
    
instance Monad (Cont r) where
    return :: a -> Cont r a
    return = pure
    (>>=) :: Cont r a -> (a -> Cont r b) -> Cont r b
    m >>= k = Cont $ \l -> m >>- \x -> k x >>- \y -> l y

## `Join` follows from `pure`

It turns out that the `Applicative` instance for `Cont` already implies the `Monad` instance. In the `Cont` monad, the `join m` is equivalent to `m . pure` (minus operations for dealing with the `runCont` boilerplate. This is illustrated below.

- TODO prove that this is equivalent to join as standardly defined.

In [32]:
join :: Cont r (Cont r a) -> Cont r a
join m = Cont $ (((>>-) <$> m) >>-) . (>>-) . pure